In [1]:
import dask.dataframe as dd
import re
import json

from dask.distributed import Client, progress
from collections import defaultdict

DATA_DIR = 'YOUR DATA DIRECTORY HERE'
DATA_DIR_FULL = DATA_DIR + "PATH TO PARQUET FILES"
Client()

Client Scheduler: tcp://127.0.0.1:60869 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 12.00 GB


## Setup

In [2]:
df = dd.read_parquet(DATA_DIR_FULL, columns=['script_url', 'symbol', 'value', 'location', 'argument_0'])
df.head()

,script_url,symbol,value,location,argument_0
0,https://mech.iitm.ac.in/meiitm/wp-includes/js/...,window.navigator.userAgent,Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko...,https://mech.iitm.ac.in/meiitm/,
1,https://mech.iitm.ac.in/meiitm/wp-includes/js/...,window.navigator.userAgent,Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko...,https://mech.iitm.ac.in/meiitm/,
2,https://mech.iitm.ac.in/meiitm/wp-admin/js/iri...,window.navigator.userAgent,Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko...,https://mech.iitm.ac.in/meiitm/,
3,https://mech.iitm.ac.in/meiitm/wp-admin/js/iri...,window.navigator.appName,Netscape,https://mech.iitm.ac.in/meiitm/,
4,https://mech.iitm.ac.in/meiitm/wp-includes/js/...,window.navigator.appVersion,5.0 (X11),https://mech.iitm.ac.in/meiitm/,


## Candidate URLs for 'CanvasRenderingContext2D.font'

In [4]:
font_df_sixth_sense = df[df.symbol == 'CanvasRenderingContext2D.font']
font_df_sixth_sense = font_df_sixth_sense[['value', 'script_url', 'location']]
font_df_sixth_sense = font_df_sixth_sense.drop_duplicates().persist()
progress(font_df_sixth_sense, notebook=False)

In [5]:
font_df_sixth_sense = font_df_sixth_sense.compute()
font_df_sixth_sense.head()

,value,script_url,location
942,18pt Tahoma,http://www.qvc.com/akam/10/2b30e194,http://www.qvc.com/for-the-home/northern-night...
1006,16pt Arial,http://www.qvc.com/_bm/async.js,http://www.qvc.com/for-the-home/northern-night...
1015,6pt Arial,http://www.qvc.com/_bm/async.js,http://www.qvc.com/for-the-home/northern-night...
2822,18pt Tahoma,http://www.coupang.com/akam/10/4f2b47,http://www.coupang.com/np/categories/331811
3399,11pt Arial,https://www.coches.net/ztkieflaaxcvaiwh121837.js,https://www.coches.net/fiat/segunda-mano/


In [6]:
%%time

num_fonts_sixth_sense = defaultdict(set)
    
for index, row in font_df_sixth_sense.iterrows():
    num_fonts_sixth_sense[(row.script_url, row.location)].add(row.value)

CPU times: user 44.7 s, sys: 654 ms, total: 45.4 s
Wall time: 45.2 s


In [7]:
font_urls_not_enough_sixth_sense = [x for x, font_set in num_fonts_sixth_sense.items() if len(font_set) < 50]
font_urls_candidates_sixth_sense = [x for x, font_set in num_fonts_sixth_sense.items() if len(font_set) >= 50]

In [8]:
print('# of unique tuples (script_url, location) with 50 or more calls:', len(font_urls_candidates_sixth_sense))
print('# of unique tuples (script_url, location) with less than 50:', len(font_urls_not_enough_sixth_sense))

# of unique tuples (script_url, location) with 50 or more calls: 2089
# of unique tuples (script_url, location) with less than 50: 58664


In [9]:
with open('not_font_fingerprinting_1.json', 'w') as f:
    f.write(json.dumps(list(set([script_url for script_url, location in font_urls_not_enough_sixth_sense]))))

## Candidate URLs for 'CanvasRenderingContext2D.measureText'

In [10]:
measure_text_df_sixth_sense = df[df.symbol == 'CanvasRenderingContext2D.measureText']
measure_text_df_sixth_sense = measure_text_df_sixth_sense[['argument_0', 'script_url', 'location']]
measure_text_df_sixth_sense['to_count'] = 1
measure_text_df_sixth_sense = measure_text_df_sixth_sense.groupby(['argument_0', 'script_url', 'location']).count().persist()
progress(measure_text_df_sixth_sense, notebook=False)

In [11]:
measure_text_df_sixth_sense = measure_text_df_sixth_sense.compute()
measure_text_df_sixth_sense = measure_text_df_sixth_sense.reset_index('argument_0', drop=True)
measure_text_df_sixth_sense.head()

to_count
script_url                                         location                                                    
https://m.stripe.network/inner.html#referrer=&t... https://m.stripe.network/inner.html#referrer=&t...        58
https://tpc.googlesyndication.com/pagead/gadget... https://googleads.g.doubleclick.net/pagead/ads?...         2
https://m.stripe.network/inner.html#url=https%3... https://m.stripe.network/inner.html#url=https%3...        16
https://s.flipboard.com/webapp/js/app-c5213e3f.js  https://flipboard.com/@libertaddigital                     1
                                                   https://flipboard.com/@libertaddigital                     1

In [12]:
measure_text_not_enough_df_sixth_sense = measure_text_df_sixth_sense[measure_text_df_sixth_sense['to_count'] < 50]

In [13]:
measure_text_df_sixth_sense = measure_text_df_sixth_sense[measure_text_df_sixth_sense['to_count'] >= 50]

In [14]:
measure_text_urls_candidates_sixth_sense = set(measure_text_df_sixth_sense.index.tolist())
measure_text_urls_not_enough_sixth_sense = set(measure_text_not_enough_df_sixth_sense.index.tolist())

In [15]:
print('# of unique tuples (script_url, location) with 50 or more calls:', len(measure_text_urls_candidates_sixth_sense))
print('# of unique tuples (script_url, location) with less than 50:', len(measure_text_urls_not_enough_sixth_sense))

# of unique tuples (script_url, location) with 50 or more calls: 2409
# of unique tuples (script_url, location) with less than 50: 2661


In [16]:
with open('not_font_fingerprinting_2.json', 'w') as f:
    f.write(json.dumps(list(set([script_url for script_url, location in measure_text_urls_not_enough_sixth_sense]))))

## Cross reference lists

In [18]:
font_urls_candidates_sixth_sense = set(font_urls_candidates_sixth_sense)

all_candidate_urls_sixth_sense = font_urls_candidates_sixth_sense | measure_text_urls_candidates_sixth_sense
canvas_font_fp_urls_sixth_sense = font_urls_candidates_sixth_sense & measure_text_urls_candidates_sixth_sense

print('# of unique tuples (script_url, location) meeting both criteria:', len(canvas_font_fp_urls_sixth_sense))
print('# of unique tuples (script_url, location) that only meet one criteria:', len(all_candidate_urls_sixth_sense - canvas_font_fp_urls_sixth_sense))

# of unique tuples (script_url, location) meeting both criteria: 1900
# of unique tuples (script_url, location) that only meet one criteria: 698


In [19]:
with open('not_font_fingerprinting_3.json', 'w') as f:
    f.write(json.dumps(list(set([script_url for script_url, location in (all_candidate_urls_sixth_sense - canvas_font_fp_urls_sixth_sense)]))))

## Save Candidate URLs

In [20]:
with open('font_fingerprinting.json', 'w') as f:
    f.write(json.dumps(list(set([script_url for script_url, location in canvas_font_fp_urls_sixth_sense]))))

## Find Locations

In [2]:
with open('font_fingerprinting.json', 'r') as f:
    font_fp_urls = json.load(f)
    
print(len(font_fp_urls), '== 1387')

1387 == 1387


In [3]:
df = dd.read_parquet(DATA_DIR_FULL, columns=['script_url', 'location'])

In [4]:
df_locs = df[df.script_url.isin(font_fp_urls)]
locs = df_locs.location.unique().persist()
progress(locs, notebook=False)

In [5]:
print('# of locations that call audio fingerprinting scripts:', len(locs))

# of locations that call audio fingerprinting scripts: 2293
